In [1]:
import os
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
os.chdir('../..')

In [2]:
def _get_day_time_df(day_time_meta_path):
    # Читаем первый лист
    meta1 = pd.read_excel(day_time_meta_path, sheet_name=0, header=1).rename(columns={
        'Subject ID'          : 'Subject_id',
        'Время начала записи' : 'Time',
    })
    meta1['Trial_id'] = 1
    
    # Читаем второй лист
    meta2 = pd.read_excel(day_time_meta_path, sheet_name=1, header=1).rename(columns={
        'Subject ID'          : 'Subject_id',
        'Время начала записи' : 'Time',
    })
    meta2['Trial_id'] = 2
    
    # Конкатенация
    meta = pd.concat([meta1, meta2], ignore_index=True)
    
    meta
    
    meta['Subject_id'] = (
        meta['Subject_id']
        .astype(str)
        .str.extract(r'(\d+)', expand=False)
        .astype('float')
    )
    meta = meta[meta['Subject_id'].notna()].copy()
    meta['Subject_id'] = meta['Subject_id'].astype(int)
    
    s  = meta['Time'].astype(str).str.strip()                    
    n  = pd.to_numeric(s, errors='coerce')                        
    dt_str = pd.to_datetime(s, errors='coerce', dayfirst=True, infer_datetime_format=True)
    dt_num = pd.to_datetime(n, errors='coerce', origin='1899-12-30', unit='D')
    dt = dt_str.fillna(dt_num)                                   
    
    meta['Hour'] = dt.dt.hour
    meta['Time'] = dt.dt.strftime('%H:%M')
    
    meta_idx = (meta[['Subject_id', 'Trial_id', 'Hour', 'Time']]
                .dropna(subset=['Hour'])                         
                .drop_duplicates(['Subject_id', 'Trial_id'], keep='last'))
    
    meta_idx['Condition'] = pd.cut(
        meta_idx['Hour'],
        bins=[-0.1, 10, 18, 24],
        labels=['Other', 'Day', 'Evening'],
        right=False,
        include_lowest=True
    ).astype(object).fillna('Other')

    return meta_idx

    


def filter_exec_psds(exec_spec_path, 
                     day_time=None,
                     day_time_meta_path=None,
                     stim_type=None, 
                     stim_label=None, 
                     gender=None, 
                     age=None, 
                     handiness=None):
    """
    Filters experimental PSD data (.npz file) based on subject and trial metadata.

    :param: exec_spec_path : str - Path to the `.npz` file containing experimental PSD data.
    :param: day_time : str - Daytime condition "Day" or "Evening"
    :param: day_time_meta_path : str - Path to an Excel file with metadata. Required only if `day_time` is provided.
    :param: stim_type : str or list(str) - "g" → geometric or "r" → random.
    :param: stim_label : int or list(int) - Label(s) of stimuli ("-1" → random or "0–12" → geometric).
    :param: gender : str - Gender "m" → male or  "f" → female.
    :param: age : int or list(int) - Age(s).
    :param: handiness : str - Hand preference "r" → right-handed or "l" → left-handed

    :return: list - filtered execution spectras [power, phase, subject_id, trial_id, gender, handiness, age, label, img, task_type].
    """
    
    loaded = np.load(exec_spec_path)
    results_arr = []

    # Load daytime metadata if filtering by daytime
    if day_time is not None:
        if day_time_meta_path is None:
            raise ValueError("You must provide 'day_time_meta_path' when 'day_time' is specified.")
        day_time_df = _get_day_time_df(day_time_meta_path)
    else:
        day_time_df = None

    # Normalize input filters
    if isinstance(stim_label, (list, np.ndarray)):
        stim_label = set(stim_label)
    if isinstance(age, (list, np.ndarray)):
        age = set(age)
    if isinstance(stim_type, str):
        stim_type = {stim_type}
    elif stim_type is not None:
        stim_type = set(stim_type)

    # Iterate through all subjects/trials in .npz file
    i = 0
    while f'power_{i}' in loaded:
        power = loaded[f'power_{i}']
        phase = loaded[f'phase_{i}']
        s_id = int(loaded[f'subject_id_{i}'])
        t_id = int(loaded[f'trial_id_{i}'])
        gend = str(loaded[f'gender_{i}'])
        hand = str(loaded[f'handiness_{i}'])
        ag = int(loaded[f'age_{i}'])
        label = int(loaded[f'label_{i}'])
        img = loaded[f'img_{i}']
        task_type = str(loaded[f'task_type_{i}'])

        # Apply filters
        # Filter by daytime condition
        if day_time is not None:
            cond_row = day_time_df[
                (day_time_df["Subject_id"] == s_id) & (day_time_df["Trial_id"] == t_id)
            ]
            if cond_row.empty or cond_row["Condition"].values[0] != day_time:
                i += 1
                continue

        # Filter by stim_type
        if stim_type is not None and task_type not in stim_type:
            i += 1
            continue

        # Filter by stim_label
        if stim_label is not None:
            if isinstance(stim_label, set):
                if label not in stim_label:
                    i += 1
                    continue
            elif label != stim_label:
                i += 1
                continue

        # Filter by gender
        if gender is not None and gend != gender:
            i += 1
            continue

        # Filter by age
        if age is not None:
            if isinstance(age, set):
                if ag not in age:
                    i += 1
                    continue
            elif ag != age:
                i += 1
                continue

        # Filter by handiness
        if handiness is not None and hand != handiness:
            i += 1
            continue

        results_arr.append([power, phase, s_id, t_id, gend, hand, ag, label, img, task_type])
        i += 1

    return results_arr

    

# Example: Get day female 18 or 30 y.o. subject execution spectras
results_arr = filter_exec_psds('./Generated/Spectrums/exec_morlets.npz',
                              day_time_meta_path="./Supplementary/Experiment_Metadata.xlsx",
                              day_time="Day",
                              gender="f",
                              age=[18, 30])

power, phase, s_id, t_id, gender, handiness, age, label, img, task_type = results_arr[0]

print(f"Power: \t {power.shape}\nPhase: \t {phase.shape}\nSubject: \t {s_id}\nTrial: \t\t {t_id}\nGender: \t {gender}\nHandiness: \t {handiness}\nAge: \t\t {age}\nTask_Type: \t {task_type}\nLabel: \t\t {label}\nImage:\n{img}")

Power: 	 (63, 195, 321)
Phase: 	 (63, 195, 321)
Subject: 	 4
Trial: 		 2
Gender: 	 f
Handiness: 	 r
Age: 		 30
Task_Type: 	 g
Label: 		 5
Image:
[[0 0 0 0 0 0]
 [1 1 1 1 1 1]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 1 1 1 1 1]
 [0 0 0 0 0 0]]
